# Select a couple of simpler trips to compare

* Compare methodologies, which differ when handling more complex shapes
* But is it different even in simpler shapes?
* Start with Big Blue Bus and LA Metro

In [27]:
import geopandas as gpd
import pandas as pd
from siuba import *

from shared_utils import rt_dates, rt_utils
from segment_speed_utils.project_vars import SEGMENT_GCS

from prep_comparison import map_one_trip

analysis_date = rt_dates.DATES["sep2023"]

In [2]:
df_eric = gpd.read_parquet(f"{SEGMENT_GCS}speeds_eric_{analysis_date}.parquet")
df_tiff = gpd.read_parquet(f"{SEGMENT_GCS}speeds_tiff_{analysis_date}.parquet")
speed_df = pd.read_parquet(
    f"{SEGMENT_GCS}speeds_comparison_{analysis_date}.parquet")

## Side-by-Side Maps

In [3]:
df_eric.shape, df_tiff.shape 

((151014, 23), (155314, 24))

In [4]:
#df_tiff.route_id.unique()

In [5]:
#one_route = "96-13168"
#df_tiff[df_tiff.route_id==one_route].explore(
#    "route_id", tiles = "CartoDB Positron")

In [6]:
#df_tiff[df_tiff.route_id==one_route].trip_id.unique()

In [7]:
trips_to_try = {
    "metro_720": "10720012750651-JUNE23", #route_id: 720-13168
    "metro_901": "10901000590843-JUNE23", #route_id: 901-13168
    "metro_550": "10550001350610-JUNE23", # route_id: 550-13168
    "metro_230":  "10230000830600-JUNE23", # route_id: 230-13168
    "metro_96": "10096002510743-JUNE23", # route_id: 96-13168
    "bbb1": "908521", # route_id: 3639
    #"bbb2": "", #route_id
}

In [8]:
# map_one_trip(df_eric, trips_to_try["metro_720"])

In [9]:
# map_one_trip(df_tiff, trips_to_try["metro_720"])

In [20]:
map_one_trip(df_eric, trips_to_try["metro_901"])

In [21]:
map_one_trip(df_tiff, trips_to_try["metro_901"])

In [12]:
# map_one_trip(df_eric, trips_to_try["metro_550"])

In [13]:
# map_one_trip(df_tiff, trips_to_try["metro_550"])

In [14]:
# map_one_trip(df_eric, trips_to_try["metro_230"])

In [15]:
# map_one_trip(df_tiff, trips_to_try["metro_230"])

In [35]:
map_one_trip(df_eric, trips_to_try["metro_96"])

In [36]:
map_one_trip(df_tiff, trips_to_try["metro_96"])

In [18]:
# map_one_trip(df_eric, trips_to_try["bbb1"])

In [19]:
# map_one_trip(df_tiff, trips_to_try["bbb1"])

# Dig deeper into a specific LA Metro G Line (901) segment

* big speed difference, but...
* not actually the same segment, Tiff is stop to stop but Eric cuts an interpolated segment since distance between stops >1000m
* interesting case where Eric _slower_ than Tiff when general pattern is opposite, but due to segment mismatch maybe not the best to look at first

In [24]:
trip_segment = {'trip_id': trips_to_try['metro_901'], 'stop_sequence': 8}

In [ ]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [31]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
109199,11377.078239,15606,Woodley Station,"LINESTRING (139447.609 -424394.891, 139508.002...",9010059_JUNE23,10901000590843-JUNE23,8.0,901-13168,None,0.0,...,377.078239,2.582728,NaN,5.777562,9,12.224986,6.458984,16.165203,2.502747,9.0


In [33]:
(df_eric >> filter_trip).explore()

In [32]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
17626,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,20c826dfb5a463334e8942cdb3e795d5,"LINESTRING (138216.368 -424448.224, 138216.382...",15606,8,0,25889,7,1ee881f8148d699747dbcd0859fcd7f6,1612.503377,...,1605.959777,259,0.995942,13.87078,0,LA Metro Bus Schedule,10901000590843-JUNE23,9010059_JUNE23,901-13168,0.0


In [34]:
(df_tiff >> filter_trip).explore()

# Dig deeper into a specific LA Metro 96 segment

* good simple example, same exact segment geom!
* Tiff: 2.93mph, Eric: 7.95mph (over 2x!)
* so... why?

In [41]:
trip_segment = {'trip_id': trips_to_try['metro_96'], 'stop_sequence': 36}

In [42]:
filter_trip = filter(_.trip_id == trip_segment['trip_id'], _.stop_sequence == trip_segment['stop_sequence'])

In [43]:
df_eric >> filter_trip

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_id,stop_sequence,route_id,route_short_name,direction_id,...,meters_from_last,speed_from_last,delay_chg_sec,speed_mph,n_trips_shp,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour
36414,16647.487169,18094,Riverside / Riverdale,"LINESTRING (162553.508 -434619.519, 162553.710...",960251_JUNE23,10096002510743-JUNE23,36.0,96-13168,96,1.0,...,184.912802,3.556015,-8.0,7.954807,4,10.440684,7.836958,15.634983,1.995032,1.333333


In [48]:
# (df_eric >> filter_trip).explore()

In [45]:
df_tiff >> filter_trip

,schedule_gtfs_dataset_key,shape_array_key,geometry,stop_id,stop_sequence,loop_or_inlining,seg_idx,district,trip_instance_key,segment_meters,...,meters_elapsed,sec_elapsed,pct_segment,speed_mph,flag_recalculated,name,trip_id,shape_id,route_id,direction_id
70379,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,7a4a0395f4918ece6eb0c18164b8c053,"LINESTRING (162553.508 -434619.519, 162553.710...",18094,36,0,93587,7,edfe9a6af705fe4181d3b916df2fdddf,184.912802,...,181.149217,138,0.979647,2.936455,0,LA Metro Bus Schedule,10096002510743-JUNE23,960251_JUNE23,96-13168,1.0


In [49]:
# (df_tiff >> filter_trip).explore()

## How does `rt_parser` calculate _this segment_??

In [50]:
from rt_analysis import rt_parser

## How does `rt_segment_speeds` calculate _this segment_??